##Deep Learning Guide
---
####By: Sean McLaughlin
####mclaughlin6464@gmail.com
----

I spent the last few weeks of my time in the LCDM trying ways to apply deep learning techniques to cosmological simulations. This was inspired by work done by group member Harshil Kamdar. Harshil was able to show that using a variety of machine learning techniques (primarily gradient boosted decision trees) one can reproduce the results of Semi-analytic Models (SAMs). Harshil's work is available [here](https://github.com/ProfessorBrunner/ml4cosmosims).

The immense success of this work motivated Prof. Brunner to look for more applications of these ideas. Specifically, he wanted to see if Deep Learning (a more powerful, cutting-edge machine learning technique) could be applied to more raw simulation data. To over-simplify a bit, Harshil's work takes certain aggregate parameters from the simulation and is able to very accurately predict other parameters. Prof. Brunner's hope was that deep learning could be directly applied to raw particle data, to predict their evolution or something else like it. It's an ambitous goal, but if it works it would be very groudbreaking. 

###1) Deep Learning Basics
---
Unfortunately, the down side of Deep Learning being "powerful" and "cutting-edge" is that it is also "complicated" and "difficult to understand and use." Its developing popularity has led to a few good online tutorials being available. Short of adding another one to the mix, I'll lay out the ones I used to get up to speed. 

1. [A Hacker's Guide to Neural Networks](http://karpathy.github.io/neuralnets/) This one has a really basic introduction to the theory. It's pretty basic and short, but a good start for when you get to the harder ones. 
2. [Deep Learning Review Paper](http://www.nature.com/nature/journal/v521/n7553/full/nature14539.html) This paper is a review of the topics in Deep Learning. A great, quick read for how the field got to where it is now, and the current cutting-edge research. 
3. [Neural Networks Class](https://www.youtube.com/playlist?list=PL6Xpj9I5qXYEcOhn7TqghAJ6NAPrNmUBH) This lecture series on YouTube is very high-level and comprehensive. I watched the whole series, but I think it wouldn't be necessary to go beyond lesson 7 (Deep Learning). 
4. [Deep Learning Tutorials in Theano](http://deeplearning.net/tutorial/) This is another high-level tutorial, which also has a few applied examples using Theano. Theano is a python package for GPU computing, which is essential to Deep Learning. It is also what I ended up using for my work, so it is a great place to learn the basics of Theano and GPU computing.

This list is of course not exhaustive. There are a number of other resources available which may be helpful. 

###2) Installation Details
---
Unlike other machine learning methods, Deep Learning is new enough and complex enough to not be implemented in popular packages like scikit-learn. There are a few implementations out there that are not as easy to install, but still way better than trying to do anything yourself.I struggled with the ins and outs of installing this software on Blue Waters. 

####Theano and libgpuarray
Theano is a python library for GPU programming. As you learned above, Neural Nets aren't really magic; instead they're just repeated applications of matrix algebra. Theano is a library that allows for constructing linear algebra calculations for executions on a GPU. 

The details of how to install theano are [here](http://deeplearning.net/software/theano/install.html). On BW most of the dependencies are taken care of, except for libgpuarray. Its details are available [here](http://deeplearning.net/software/libgpuarray/installation.html). It's possible to install theano first, so I'll start with that. 

---
***Aside:*** If you don't want to waste a bunch of BW time, you could also install these packages on your own machine, provided you have the proper hardware. If you have a powerful NVIDIA brand GPU, it may be worth a shot. NVIDIA GPUs use the CUDA framework, which is the standard in GPU computing. Theano is optimized for work with CUDA, and the GPUs on BW are NVIDIA Keplers. If you have an AMD GPU, good luck. I myself have a fairly beefy AMD GPU with 3GB of VRAM. However, I really struggled getting OpenCL running on my system, to the point that I gave up. Theano has very limited support for OpenCL anyway, so it may not have worked anyway.

---

Following along with the Theano installation guide above, I have a few addendums that I discovered on BW.

1. Unfortunately, this code cannot be run/installed in BW python enviornment, or even in the PrgEnv-gnu one. This is because the python module does not include python-dev or python-devel, which is required for compiling C backends. I complained to BW support about this and they basically looked down on me for using python. 
2. Pip isn't installed on BW, but setuptools is. To install theano, run `easy_install --user theano`. The `--user` flag ensure the code is only installed for your account, because you obviously don't have root access to the machine. 
3. I discovered very near the end of my time that by loading in anaconda via `module load anaconda` one could circumvent the python-dev issue. I didn't perform extensive testing, but I was able to run a few test jobs with the full bw-python suite in the PrgEnv-gnu enviornment. 

That's all for theano. It will not do anything intersting though until its dependencies have also been installed. Follwing along with the libgpuarray guide above:

1. BW does have CUDA on it. You will need to load it via `module load cudatoolkit`. You will also need CMake, which can be loaded via `module load cmake`. 
2. Make sure to follow the second set of commands, toward the bottom of the page. It is for installing on a system where you don't have root access.
3. CMake requires a custom install directory. I used `~/bin`, but the guide suggests `~/.local`. That may also work.
4. The `CMakeCache.txt` has to be modified. In that file that where will be a variable `CUDA_CUDA_LIBRARY` that must point to the libcuda.so file. I had to dig for it, but I found it located at `/opt/nvidia/cudatoolkit6.5/ 6.5.14-1.0502.9613.5.1/lib64/stubs/libcuda.so`. I imagine your location will be similar if not identical.
5. In order to sucessfully "`make`" the distribution, you will need to switch to the Gnu Programming enviornment. It is the only one that has GCC, which is the only compiler that will actually work. To do this, just run `module swap PrgEnv-cray PrgEnv-gnu`.
6. I vaguely remember having another problem with `make instal`l, but at this time I can't recall what it was. If you encounter it, you can contact me and I may recall what I did given more details.
7. For `setup.py build`, make sure you use the second option where you specify where you chose to install libgpuarray a few steps above. 
8. Make sure you use `--user` when running `setup.py install`.

It's possible to test your theano installation by running the following. However, in my experience it takes a long time to do, and it may just be better to try something!

`>>> import theano`

`>>>theano.test()`

On [this page](http://deeplearning.net/software/theano/tutorial/using_gpu.html) there's an example function to test GPU usage, which you can try on BW. To test it, you need to load up a CMM interactive job, otherwise you won't be able to access the GPUs. The details of how to start such a job are [here](https://bluewaters.ncsa.illinois.edu/cluster-compatibility-mode). Make sure you load an xk node, as those are the only ones with accelerators. Also make sure you read all the way down on that page to the `module add ccm;ccmlogin` portion to actually start the CCM session. 

An important thing to note when using theano is its use of flags. In the example linked above, the use of the `device` flag sets which device to use. For my jobs on BW, I used the following declarations in bash (or in a .sh file).

In [1]:
%%bash
device = gpu0
export THEANO_FLAGS=mode=FAST_RUN,device=$device,floatX=float32 #,blas.ldflags=-lsci-cray

bash: line 1: device: command not found


I occasionally ran into trouble with theano not being able to find the BLAS package it wanted. After ensuring that libsci-cray is loaded on BW, uncommenting that last part sets the flag with where to find it. I found that for PDNN it would accept the flag I passed in, but for pylearn2 it would prefer scipy's package anyway (hence the commenting out). 

####PDNN

PDNN is a python deep learning framework built on top of theano. Esentially, someone took theano's nuts and bolts and built something that actually has layer objects and training methods, so you don't have to build it yourself. [Their webpage](http://www.cs.cmu.edu/~ymiao/pdnntk.html) has details on how to run and install it, and links to a few tutorials. Installing it on BW is blissfully very simple. All it takes is a single run of 

`git clone <url>`

of the git url and it's done. Let me say that before you install it from them, you should read further to see why you should probably clone my fork of it, since I made some critical edits. You will need to also add the directory to the PYTHONPATH. They suggest trying 

`export PYTHONPATH=/path/to/pdnn:$PYTHONPATH`

in terminal. This didn't always work for me (even if I added it to my .bashrc file) so in my run.sh file, I'd just add the line before the execution calls. 

I initially picked up PDNN because on the surface it seemed easier to pick up than pylearn2. That turned out to be incorrect, but I'll discuss it briefly in any case. The biggest trouble was that PDNN is built only to do classification, not regression. All outputs were run through a softmax function which forces them all to sum to 1 (reflecting probabilities) and the loss function was calculating missclassificaitons rather than negative log liklihood like one would prefer for regression. I had to manually edit a few of the objects in order to do regression. In the end the edits weren't too difficult, but thenao code turns out to be very tricky to debug since it JIT compiles while the python runs. If you are interested in using PDNN for regression, clone/fork [my repo.](https://github.com/mclaughlin6464/pdnn) Most of my stuff is in the cleverly named myStuff folder. I left a few object modifications in the folders I copied them from, because I am not good at coding standards. If you execute the run.sh file in that folder, it will learn on the data Harshil used in his project. You may need to edit a few filepaths, and download Harshil's data from the repo linked to above. 

####pylearn2
Pylearn2 is a very similar to PDNN, at least at a high level. [Their site](http://deeplearning.net/software/pylearn2/) gives details about it. It also is a python library that combines the guts of theano into an OOP deep learning framework. The details are provided to the program via a .yaml file. YAML stands for "Yet Another Markup Language," and though it seems pretty foreign at first it's pretty easy to grasp after playing with it for a bit. You will need to install, in addition to the packages above, PyYAML. On BW, this is easily done via:

`easy_install --user pyyaml`

The YAML syntax provides a pretty high level of control over the model. For an introduction, make sure to check out their tutorials for [softmax regression](https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/scripts/tutorials/softmax_regression/softmax_regression.ipynb) and [the multilayer perceptron](https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/scripts/tutorials/multilayer_perceptron/multilayer_perceptron.ipynb) in that order. There are a few other tutorials for deep learning and unsupervised pre-training that I have yet to study but will undoubtably be useful, too. 

Like with PDNN above, [I've also forked](https://github.com/mclaughlin6464/pylearn2) and made changes to their code, which I reccomend you clone. Pylearn2 has regression baked in, which is immensely helpful. However, one does has to build an interface to read in their data, and a few simple scripts to parse their .yaml file. Once again, my edits are all in the myStuff folder (all of them this time, I promise!). 

To test it out, try running the run_milli_sam.py and test_milli_sam.py programs. They will test out the same data. I also have code for doing regression with particles from a simulation I got from Robert Thompson. It's still not quite working right for more reasons than I can really get into here. 

###3) Preliminary Findings
---

I didn't get much of an opportunity to test things out. However, the results I did get were interesting and worth sharing. For replicating Harshil's results with random forests, I have a notebook in this directory that shows the results. It would seem that the MLP performs about as well and as fast with little tuning. With more advanced tuning of the hyperparameters, it's possible that it may be able to outperform tree-based algorithms. However, Harshil has yet to perform that tuning on his algorithms himself, so it may not be too fruitful at his point. 

I next had a very short period of time where I attempted to apply this technique to particle data. I got from Bobby Thompson a simulation with ~200,000 particles and 100 snapshots. This is a fairly small number of particles and a large number of snapshots, but it's my first attempt toward something larger. The algorithm is designed so the net takes a certain number of particles position and velocity and predicts their position at the next snapshot. This is only ~90 training examples, as ~10 need to be withdrawn for validation and testing. The algorithm can accept an arbitrary "slice" of particles, but this changes the size of the net. In effect, it reduces the size of the parameter space, not the number of data points (as each snapshot is one data point in a very high dimensional parameter space). 

I tried a number of varied paramters, from hidden unit size to number of particles to momentum and regularization. I believe there just isn't enough training data, as the predictions just do not work. The net gets biased to the training set, and the validation doesn't decrease much. One would hope that the model would learn that the new position is just the curent position plus the velocity times a constant (time step and unit conversion), ignoring acceleration which would at least be close. However, that doesn't seem to be the case. There are a number of ways that dataset could be artificially inflated, like by adding "fuzz" to the initial values and by rotating the coordinate system around. I still have doubts that it would be enough.